# COVID -19 Exploratory Analysis

## COVID-19_ADT


In [2]:
#start session
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2')
#from pyspark import SparkContext
#from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import SparkSession
from timeit import default_timer as timer
#from modules import claims_sql
from pyspark.sql.functions import col, round

import sys
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
#DW Netezza Login Information
import getpass
import json
from datetime import date, datetime
import pandas as pd
pd.set_option('display.max_columns', None,'display.max_columns', None)

from IPython.core.interactiveshell import InteractiveShell #display results setting
InteractiveShell.ast_node_interactivity = "all"

uid = input("Enter LAN ID: ")
pwd = getpass.getpass("Enter LAN pwd: ")

Enter LAN ID: abaner02
Enter LAN pwd: ········


In [3]:
spark = SparkSession \
        .builder \
        .appName("COVID-19_ADT") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.memory", "48G") \
        .config("spark.executor.cores", "5") \
        .getOrCreate()


In [1]:
spark.stop()

NameError: name 'spark' is not defined

In [3]:
sql = '''select * from cln.V_BCBSMA_ADT_CMT_FNL'''
df = spark.sql(sql)

In [4]:
df.count()

120677

In [5]:
df.show()

+-------------------+-----------+------------------+-----------------+----------------------+-------------------+--------------------+----------------------+-------------+-------------+----------------+--------------------+--------------+--------------------+--------------------+----------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------+-------------------------------+------------------------+---------------------+-----------------+--------------------+------------------------+------------------------+-------------+-------------------------+--------------------+-------------------------+-----------------------------+-------------------------------+----------------------------+------------------------+--------------------+-------------+--------------+-------------------+-----------------+----------------------+---------------------+---------------------+---------------------------------+---------------------

In [6]:
df_p = df.toPandas()

In [7]:
df_p.head()

curr_timestamp    member_id patient_first_name patient_last_name  \
0 2019-11-01 07:18:28  98392890400            WILLIAM          LOUGHLIN   
1 2019-11-03 08:16:39  98284784200               JANE              DION   
2 2019-08-12 07:15:17  98249037100         JACQUELINE        ROBITAILLE   
3 2019-08-12 07:15:49  98071877001              BARRY           JACKSON   
4 2019-07-28 07:15:22  98119619410             CONNOR            GORHAM   

     patient_formatted_name         patient_dob  \
0       LOUGHLIN, WILLIAM J 1934-06-23 04:00:00   
1              DION, JANE E 1959-08-14 04:00:00   
2  ROBITAILLE, JACQUELINE L 1948-01-30 05:00:00   
3            JACKSON, BARRY 1957-01-17 05:00:00   
4            GORHAM, CONNOR 2001-05-01 04:00:00   

                            patient_full_address patient_street_address  \
0            69 SHAWSHEEN RD, LAWRENCE, MA 01843        69 SHAWSHEEN RD   
1  465 MILLERS FALLS RD, MILLERS FALLS, MA 01349   465 MILLERS FALLS RD   
2             5 BEECHWOOD DR, ACUSHNET, MA 02743         5 BEECHWOOD DR   
3        53 BIRCHMEADOW ROAD, MERRIMAC, MA 01860    53 BIRCHMEADOW ROAD   
4          884 MAIN STREET, MARSHFIELD, MA 02050        884 MAIN STREET   

    patient_city patient_state patient_zip_code patient_phone_number  \
0       LAWRENCE            MA            01843       (978) 857-4728   
1  MILLERS FALLS            MA            01349       (413) 423-3231   
2       ACUSHNET            MA            02743       (508) 995-7089   
3       MERRIMAC            MA            01860       (978) 346-9530   
4     MARSHFIELD            MA            02050       (781) 834-1088   

  patient_gender                            patient_id  \
0           Male  d01d5a31-8753-43c8-8c8d-f577c278021f   
1         Female  e0d1d3f0-4e75-4a79-a0c8-3a0d501a4d4b   
2         Female  12172a72-32f8-4ac5-94b7-2992f22bc351   
3           Male  d291370e-0f30-4275-8d9b-e7ce11c35207   
4           Male  975680e3-033c-426e-8a47-d7ecd5267f01   

               subscriber_id patient_deceased  \
0  983928904-19340623-WILL-M             None   
1  982847842-19590814-JANE-F             None   
2  982490371-19480130-JACQ-F             None   
3  980718770-19570117-BARR-M             true   
4  981196194-20010501-CONN-M            false   

            encounter_facility_name encounter_facility_street  \
0                    Prescott House       140 Prescott Street   
1  Baystate Franklin Medical Center               164 High St   
2               St. Luke's Hospital           101 Page Street   
3    Holy Family Hospital - Methuen               70 East St.   
4              South Shore Hospital              55 Fogg Road   

  encounter_facility_city encounter_facility_state encounter_facility_zip  \
0           North Andover                       MA                  01845   
1              Greenfield                       MA                  01301   
2             New Bedford                       MA                  02740   
3                 Methuen                       MA                  01844   
4          South Weymouth                       MA                  02190   

                encounter_facility_full_address encounter_facility_phone  \
0  140 Prescott Street, North Andover, MA 01845           (978) 685-8086   
1             164 High St, Greenfield, MA 01301           (413) 773-0211   
2        101 Page Street, New Bedford, MA 02740           (508) 997-1515   
3               70 East St. , Methuen, MA 01844           (978) 687-0151   
4        55 Fogg Road, South Weymouth, MA 02190           (781) 624-8000   

  encounter_major_class   encounter_type encounter_admit_date  \
0       Post Acute Care  Skilled Nursing  2019-10-31 04:00:00   
1             Emergency        Emergency  2019-11-02 04:00:00   
2             Emergency        Emergency  2019-08-10 04:00:00   
3             Emergency        Emergency  2019-08-11 04:00:00   
4             Emergency        Emergency  2019-07-27 04:00:00   

  encounter_dischar

In [10]:
sql = '''
select distinct 
encounter_diagnosis,
encounter_cheif_complaint,
encounter_diagnosis_icd10,
encounter_type,
encounter_discharge_disposition,
encounter_admitted_inpatient,
encounter_facility_name,
encounter_facility_city,
encounter_facility_zip,
encounter_account_number,
count(distinct member_id)
from cln.V_BCBSMA_ADT_CMT_FNL
where 
--encounter_diagnosis like '*COVID*' or encounter_cheif_complaint like '*COVID*'
encounter_diagnosis like '%COVID%' or encounter_cheif_complaint like '%COVID%' 
or encounter_diagnosis like '%covid%' or encounter_cheif_complaint like '%covid%' 
or encounter_diagnosis like '%CORONAV%' or encounter_cheif_complaint like '%CORONAV%' 
or encounter_diagnosis like '%coronav%' or encounter_cheif_complaint like '%coronav%' 
or encounter_diagnosis like '%CORONA V%' or encounter_cheif_complaint like '%CORONA V%' 
or encounter_diagnosis like '%corona v%' or encounter_cheif_complaint like '%corona v%' 
group by
encounter_diagnosis,
encounter_cheif_complaint,
encounter_diagnosis_icd10,
encounter_type,
encounter_discharge_disposition,
encounter_admitted_inpatient,
encounter_facility_name,
encounter_facility_city,
encounter_facility_zip,
encounter_account_number
'''
df = spark.sql(sql)
df.count()

12

In [11]:
df_p1 = df.toPandas()
df_p1

encounter_diagnosis  \
0                                    COVID 19 testing   
1                                                None   
2                                     COVID19 testing   
3                                          Eval-COVID   
4                    Cough; requests COVID-19 testing   
5   R05:Cough; R509:Fever, unspecified; Cough; Sor...   
6   Cough; covid 19 exposure; potential Covid 19 e...   
7              corona exposure; corona virus exposure   
8                                 Eval - COVID clinic   
9                                 Eval - COVID clinic   
10          coronavirus symptoms; Coronavirus testing   
11  + BLOOD CULTURES,IMMUNOSUPPRESSED,COVID PRECAU...   

                  encounter_cheif_complaint encounter_diagnosis_icd10  \
0                                      None                      None   
1   COPD EXACERBATION CORONAVIRUS INFECTION                      None   
2                                      None                      None   
3                                      None                      None   
4                                      None                         ;   
5                                      None          R05; R509; ; ; ;   
6                                      None                       ; ;   
7                                      None                         ;   
8                                      None                      None   
9                                      None                      None   
10                                     None                         ;   
11                                     None                      None   

      encounter_type                    encounter_discharge_disposition  \
0          Emergency                                 Diverted Elsewhere   
1   General Medicine  Discharged/transferred to home under care of o...   
2          Emergency                                 Diverted Elsewhere   
3          Emergency  Discharged to home or self care (routine disch...   
4          Emergency                                 Diverted Elsewhere   
5          Emergency  Discharged to home or self care (routine disch...   
6          Emergency                                 Diverted Elsewhere   
7          Emergency                                          Elopement   
8          Emergency  Discharged to home or self care (routine disch...   
9          Emergency  Discharged to home or self care (routine disch...   
10         Emergency  Discharged to home or self care (routine disch...   
11  Medical Surgical                                               None   

   encounter_admitted_inpatient         encounter_facility_name  \
0                            No  Massachusetts General Hospital   
1                           Yes         Baystate Medical Center   
2                            No  Massachusetts General Hospital   
3                            No  Massachusetts General Hospital   
4                            No  Massachusetts General Hospital   
5                            No  Massachusetts General Hospital   
6                            No  Massachusetts General Hospital   
7                            No  Massachusetts General Hospital   
8                            No  Massachusetts General Hospital   
9                            No  Massachusetts General Hospital   
10                           No       Newton-Wellesley Hospital   
11                          Yes            Anna Jaques Hospital   

   encounter_facility_city encounter_facility_zip encounter_account_number  \
0                   Boston                  02114               3298779250   
1              Springfield                  01199               0850917443   
2                   Boston                  02114               3298797380   
3                   Boston                  02114               3299127603   
4                   Boston                  02114               3299261970   
5                   Boston

In [27]:
sql = '''
select distinct 
encounter_diagnosis
from cln.V_BCBSMA_ADT_CMT_FNL

'''
df_diagnosis = spark.sql(sql).toPandas()
df_diagnosis.count()

encounter_diagnosis    59848
dtype: int64

In [12]:
sql = '''
select distinct encounter_account_number from from cln.V_BCBSMA_ADT_CMT_FNL

'''
acc = spark.sql(sql).toPandas()
len(acc)

92448

In [113]:
sql = '''
(select * from V_BCBSMA_ADT_CMT_FNL) a'''
#uid = 'abaner02'
df = spark \
        .read \
        .format("jdbc") \
        .option("url", "jdbc:netezza://bntzp01z.bcbsma.com:5480/PDWAPPRP") \
        .option("user", uid) \
        .option("password", pwd) \
        .option("driver", "org.netezza.Driver") \
        .option("dbtable", sql) \
        .load()

df.show()

+-------------------+-----------+------------------+-----------------+----------------------+-----------+--------------------+----------------------+----------------+-------------+----------------+--------------------+--------------+--------------------+--------------------+----------------+-----------------------+-------------------------+-----------------------+------------------------+----------------------+-------------------------------+------------------------+---------------------+----------------+--------------------+------------------------+------------------------+-------------+-------------------------+--------------------+-------------------------+-----------------------------+-------------------------------+----------------------------+------------------------+--------------------+-------------+--------------+-------------------+-----------------+----------------------+---------------------+---------------------+---------------------------------+--------------------------+

In [114]:
df.count()

122830

In [115]:
df.createOrReplaceTempView('ADT_DATA')

In [117]:
# load county - zip mapping
df_county_zip = pd.read_csv('county_zip_code_list.csv',sep='|',dtype={'Zip':'str'})
spark.createDataFrame(df_county_zip).createOrReplaceTempView('COUNTY_ZIP')
df_county_zip.head()

Zip     City     County
0  01001   Agawam    Hampden
1  01002  Amherst  Hampshire
2  01003  Amherst  Hampshire
3  01004  Amherst  Hampshire
4  01005    Barre  Worcester

In [118]:
sql = '''
select a.*,
    SUBSTRING(a.CURR_TIMESTAMP, 1, 10) AS yyyy_mm_dd,
    SUBSTRING(a.CURR_TIMESTAMP, 1, 4) AS yyyy,
    SUBSTRING(a.CURR_TIMESTAMP, 1, 7) AS yyyy_mm,
    SUBSTRING(a.CURR_TIMESTAMP, 6, 5) AS mm_dd,
    b.County as patient_county,
    c.County as facility_county
    from ADT_DATA as a 
        left join COUNTY_ZIP b
            on a.PATIENT_ZIP_CODE = b.Zip
        left join COUNTY_ZIP c
            on a.ENCOUNTER_FACILITY_ZIP = c.Zip
'''
df = spark.sql(sql)
df.createOrReplaceTempView('ADT_DATA_COUNTY')
df.write.mode("overwrite").format('parquet').saveAsTable('analytics.w5_covid_19_ADT_Netezza_'+uid)

In [119]:
df.count()

122830

In [120]:
sql = '''
select 
    yyyy_mm_dd,mm_dd,--facility_county,
    patient_county,count(distinct member_id) as mem_cnt
    from ADT_DATA_COUNTY 
    where yyyy_mm = '2020-03'
    group by yyyy_mm_dd,mm_dd,--facility_county,
    patient_county 
    order by 1 DESC
'''
df_county_grp = spark.sql(sql).toPandas()
df_county_grp.to_csv('county_level_ADT_by_date.csv')
len(df_county_grp) , df_county_grp.head()

(264,    yyyy_mm_dd  mm_dd patient_county  mem_cnt
 0  2020-03-24  03-24        Hampden        9
 1  2020-03-24  03-24        Norfolk       19
 2  2020-03-24  03-24       Plymouth       10
 3  2020-03-24  03-24          Essex       13
 4  2020-03-24  03-24      Worcester       15)

In [121]:
sql = '''
select 
    mm_dd,--facility_county,
    patient_county,count(distinct member_id) as covid_mem_cnt
    from ADT_DATA_COUNTY 
    where yyyy_mm = '2020-03' and
    encounter_diagnosis like '%COVID%' or encounter_cheif_complaint like '%COVID%' 
    or encounter_diagnosis like '%covid%' or encounter_cheif_complaint like '%covid%' 
    or encounter_diagnosis like '%CORONAV%' or encounter_cheif_complaint like '%CORONAV%' 
    or encounter_diagnosis like '%coronav%' or encounter_cheif_complaint like '%coronav%' 
    or encounter_diagnosis like '%CORONA V%' or encounter_cheif_complaint like '%CORONA V%' 
    or encounter_diagnosis like '%corona v%' or encounter_cheif_complaint like '%corona v%' 
    group by mm_dd,--facility_county,
    patient_county 
    order by 1 DESC
'''
df_county_grp_cvid = spark.sql(sql).toPandas()
df_fin = df_county_grp.merge(df_county_grp_cvid, how='left',on=['mm_dd','patient_county']).fillna(0)
df_fin.to_csv('county_level_ADT_by_date_covid.csv')
len(df_fin) , df_fin.head()

(264,    yyyy_mm_dd  mm_dd patient_county  mem_cnt  covid_mem_cnt
 0  2020-03-24  03-24        Hampden        9            0.0
 1  2020-03-24  03-24        Norfolk       19            0.0
 2  2020-03-24  03-24       Plymouth       10            0.0
 3  2020-03-24  03-24          Essex       13            2.0
 4  2020-03-24  03-24      Worcester       15            1.0)

In [126]:
sql = '''
select distinct
    ENCOUNTER_FACILITY_NAME,
    ENCOUNTER_FACILITY_STREET,
    ENCOUNTER_FACILITY_CITY,
    ENCOUNTER_FACILITY_STATE,
    ENCOUNTER_FACILITY_ZIP,
    facility_county,
    yyyy_mm_dd,mm_dd,
    count(distinct member_id) as mem_cnt
from ADT_DATA_COUNTY
   where yyyy = '2020' and ENCOUNTER_FACILITY_STATE = 'MA'
   group by
   ENCOUNTER_FACILITY_NAME,
    ENCOUNTER_FACILITY_STREET,
    ENCOUNTER_FACILITY_CITY,
    ENCOUNTER_FACILITY_STATE,
    ENCOUNTER_FACILITY_ZIP,
    facility_county,
    yyyy_mm_dd,mm_dd
'''
df=spark.sql(sql)
df.write.mode("overwrite").format('parquet').saveAsTable('analytics.w5_covid_19_ADT_Facilities_'+uid)
df.toPandas().to_csv('facilties_county_level_ADT_by_date.csv')
len(df.toPandas()), df.toPandas().head()

(3931,          ENCOUNTER_FACILITY_NAME ENCOUNTER_FACILITY_STREET  \
 0  Nashoba Valley Medical Center           200 Groton Road   
 1  Nashoba Valley Medical Center           200 Groton Road   
 2  Nashoba Valley Medical Center           200 Groton Road   
 3  Nashoba Valley Medical Center           200 Groton Road   
 4  Nashoba Valley Medical Center           200 Groton Road   
 
   ENCOUNTER_FACILITY_CITY ENCOUNTER_FACILITY_STATE ENCOUNTER_FACILITY_ZIP  \
 0                    Ayer                       MA                  01432   
 1                    Ayer                       MA                  01432   
 2                    Ayer                       MA                  01432   
 3                    Ayer                       MA                  01432   
 4                    Ayer                       MA                  01432   
 
   facility_county  yyyy_mm_dd  mm_dd  mem_cnt  
 0       Middlesex  2020-01-19  01-19        5  
 1       Middlesex  2020-01-20  01-20        3

In [7]:
sql = '''
select distinct 
encounter_diagnosis,
encounter_cheif_complaint,
encounter_diagnosis_icd10,
encounter_type,
encounter_discharge_disposition,
encounter_admitted_inpatient,
encounter_facility_name,
encounter_facility_city,
encounter_facility_zip,
encounter_account_number,
count(distinct member_id)
--from cln.V_BCBSMA_ADT_CMT_FNL
from ADT_DATA
where 
--encounter_diagnosis like '*COVID*' or encounter_cheif_complaint like '*COVID*'
encounter_diagnosis like '%COVID%' or encounter_cheif_complaint like '%COVID%' 
or encounter_diagnosis like '%covid%' or encounter_cheif_complaint like '%covid%' 
or encounter_diagnosis like '%CORONAV%' or encounter_cheif_complaint like '%CORONAV%' 
or encounter_diagnosis like '%coronav%' or encounter_cheif_complaint like '%coronav%' 
or encounter_diagnosis like '%CORONA V%' or encounter_cheif_complaint like '%CORONA V%' 
or encounter_diagnosis like '%corona v%' or encounter_cheif_complaint like '%corona v%' 
group by
encounter_diagnosis,
encounter_cheif_complaint,
encounter_diagnosis_icd10,
encounter_type,
encounter_discharge_disposition,
encounter_admitted_inpatient,
encounter_facility_name,
encounter_facility_city,
encounter_facility_zip,
encounter_account_number
'''
df = spark.sql(sql)
df.count()

43

In [8]:
df_p1_n = df.toPandas()
df_p1_n

encounter_diagnosis  \
0                                    Cough; r/o covid   
1              corona exposure; corona virus exposure   
2   R05:Cough; R509:Fever, unspecified; Cough; Sor...   
3   COVID EXPOSURE; B349:Viral infection, unspecified   
4                                                       
5   covid testing; J069:Acute upper respiratory in...   
6   J45901:Unspecified asthma with (acute) exacerb...   
7                    Cough; requests COVID-19 testing   
8                                     COVID19 testing   
9                          R05:Cough; COVID19 TESTING   
10         Fatigue; Shortness of Breath; R/O COVID-19   
11       Cough; contact with someone covid, pneumonia   
12  Cough; covid 19 exposure; potential Covid 19 e...   
13                                Eval - COVID clinic   
14  Hemoptysis; Shortness of Breath; B9729:Oth cor...   
15  R0609:Other forms of dyspnea; Chest Pain; COVI...   
16  B9729:Oth coronavirus as the cause of diseases...   
17                                Eval - COVID clinic   
18                                      covid 19 test   
19  R109:Unspecified abdominal pain; covid concern...   
20  + BLOOD CULTURES,IMMUNOSUPPRESSED,COVID PRECAU...   
21  J45901:Unspecified asthma with (acute) exacerb...   
22  R05:Cough; COVID?/RESPIRATORY SYMPTOMS; J069:A...   
23                                                      
24  Z20828:Contact w and exposure to oth viral com...   
25  TRAC- CHEST PAIN, R/O COVID; SOB; Fever; B349:...   
26  J45901:Unspecified asthma with (acute) exacerb...   
27  J069:Acute upper respiratory infection, unspec...   
28  Nasal Congestion; COVID?; J069:Acute upper res...   
29  R109:Unspecified abdominal pain; covid concern...   
30  B9729:Oth coronavirus as the cause of diseases...   
31  + BLOOD CULTURES,IMMUNOSUPPRESSED,COVID PRECAU...   
32                                                      
33                        R05:Cough; COVID19- TESTING   
34  R509:Fever, unspecified; R197:Diarrhea, unspec...   
35                                                      
36                                         Eval-COVID   
37                                   COVID 19 testing   
38                                   r/o covid; Cough   
39  TRAC- CHEST PAIN, R/O COVID; SOB; Fever; B349:...   
40                                                      
41          coronavirus symptoms; Coronavirus testing   
42  B9729:Oth coronavirus as the cause of diseases...   

                  encounter_cheif_complaint  encounter_diagnosis_icd10  \
0                                                                    ;   
1                                                                    ;   
2                                                     R05; R509; ; ; ;   
3                                                               ; B349   
4   COPD EXACERBATION CORONAVIRUS INFECTION                              
5                                                               ; J069   
6                                                    J45901; J45901; ;   
7                                                                    ;   
8                                                                        
9                                                                 R05;   
10                                                                 ; ;   
11                                                                   ;   
12                                                                 ; ;   
13                                                                       
14                                                    ; ; B9729; ; J22   
15                                                          R0609; ; ;   
16                                           B9729; J22; B9729; ; J22;   
17                                                                       
18                                                                       
19                                           

In [40]:
#Fetch word count for each abstract
#PATIENT_DOB|PATIENT_FULL_ADDRESS|PATIENT_STREET_ADDRESS|PATIENT_CITY|PATIENT_STATE|PATIENT_ZIP_CODE|PATIENT_PHONE_NUMBER|PATIENT_GENDER|          PATIENT_ID|       SUBSCRIBER_ID|PATIENT_DECEASED
#DXCG_RISK_SCORE|PMI_RISK_SCORE|IMPACT_PRO_RISK_SCORE|UNABLE_TO_REACH|LANDMARK_ELIGIBLE|AQC_PROVIDER_IDENTIFIER|CASE_MANAGER|EPISODE_STATUS|ENCOUNTER_DISCHARGE_EVENT|TOP_30_DIAGS|TOP_40_DIAGS|BH_DIAGS|MEMBER_AGE|NEXT_OF_KIN|SECURITY_RISK|GAP_IN_CARE|ED_BOARDING_ALERT|SERVICING_PROVIDER_NPI_NUMBER|BILLING_PROVIDER_NPI_NUMBER|EMERGENT_OR_NON_EMERGENT|DEATH_DATE|FILE_YYYYMMDD| LOAD_DATE|

cols = ['curr_timestamp',
        'member_age','patient_city','patient_state','patient_zip_code','patient_gender','patient_dob',
        'encounter_diagnosis','encounter_cheif_complaint',
        'encounter_diagnosis_icd10','encounter_type','encounter_discharge_disposition','encounter_discharge_event',
        'encounter_admitted_inpatient','encounter_facility_name','encounter_facility_city',
        'encounter_facility_zip','encounter_account_number','episode_status',
        'TOP_30_DIAGS','TOP_40_DIAGS','BH_DIAGS','DXCG_RISK_SCORE','PMI_RISK_SCORE','IMPACT_PRO_RISK_SCORE',
        'SERVICING_PROVIDER_NPI_NUMBER','BILLING_PROVIDER_NPI_NUMBER','LOAD_DATE'
       ]
df_txt= df.select(*cols).toPandas()
df_txt.head()

curr_timestamp member_age patient_city patient_state patient_zip_code  \
0 2019-08-08 03:21:49       None       WOBURN            MA            01801   
1 2019-10-03 03:21:29       None     BRADFORD            MA            01835   
2 2019-08-12 03:15:33       None     MERRIMAC            MA            01860   
3 2019-09-05 03:22:43       None      EASTHAM            MA            02642   
4 2020-01-02 05:15:38       None   SHREWSBURY            MA            01545   

  patient_gender patient_dob  \
0         Female  1950-06-23   
1         Female  1966-05-21   
2           Male  1957-01-17   
3         Female  1942-09-11   
4         Female  1921-09-20   

                                 encounter_diagnosis  \
0  J9690:Respiratory failure, unspecified, unspec...   
1                                                      
2                                                      
3                    R SHOULDER RTC TEAR ARTHROPATHY   
4                       R627:Adult failure to thrive   

  encounter_cheif_complaint encounter_diagnosis_icd10      encounter_type  \
0                                               J9690       Critical Care   
1                                                      Ambulatory Surgery   
2                                                              Cardiology   
3                                                              Orthopedic   
4                     R62.7                      R627           Inpatient   

                     encounter_discharge_disposition  \
0                                                      
1                                                      
2                                                      
3                                                      
4  Discharged/transferred to skilled nursing faci...   

  encounter_discharge_event encounter_admitted_inpatient  \
0                                                    Yes   
1                                                    Yes   
2                                                    Yes   
3                                                    Yes   
4                         D                          Yes   

                 encounter_facility_name encounter_facility_city  \
0                  Boston Medical Center                  Boston   
1         St. Elizabeth's Medical Center                Brighton   
2         St. Elizabeth's Medical Center                Brighton   
3  Brigham and Women's Faulkner Hospital                  Boston   
4          UMass Memorial Medical Center               Worcester   

  encounter_facility_zip encounter_account_number episode_status TOP_30_DIAGS  \
0                  02118               8025714562   Not Assigned                
1                  02135             SE0101525467   Not Assigned                
2                  02135             SE0101175800   Not Assigned                
3                  02130               3264382237   Not Assigned                
4                  01605              12105545029   Not Assigned                

  TOP_40_DIAGS BH_DIAGS DXCG_RISK_SCORE PMI_RISK_SCORE IMPACT_PRO_RISK_SCORE  \
0                         DxCG: Chronic                                10.00   
1                             DxCG: Low                                 1.00   
2                         DxCG: Chronic                                 3.00   
3                             DxCG: Low     PMI: 41-50                  3.00   
4                         DxCG: At Risk      PMI: 6-10                  5.00   

  SERVICING_PROVIDER_NPI_NUMBER BILLING_PROVIDER_NPI_NUMBER   LOAD_DATE  
0                                                            2019-08-08  
1                                                            2019-10-03  
2                                                            2019-08-12  
3                                                            2019-09-05  
4                    1831151455                              2020-01-02

In [ ]:
df_txt
#https://en.wikipedia.org/wiki/User:Michael_J/County_table

In [4]:
sql = '''
    (select * from ADMIN.V_BCL_MONTHLY_CALL ) a'''
#uid = 'abaner02'
df = spark \
        .read \
        .format("jdbc") \
        .option("url", "jdbc:netezza://bntzp01z.bcbsma.com:5480/PDWAPPRP") \
        .option("user", uid) \
        .option("password", pwd) \
        .option("driver", "org.netezza.Driver") \
        .option("dbtable", sql) \
        .load()

df.createOrReplaceTempView('BCL_DATA')
df.show()

+----------------+--------------------+--------+----------+-----------------+---------------+-------------------+------------------+----------------+-------------+-------------+-------------+-----------------+-----------------+---------------+--------------------+----------+---------------+----------------+--------------+------+---------+----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------------+----------+------------------+--------------------+---------+---------------+--------------+-------------+-------------+---------+-------+------------+---------+---------+-----------+--------------------+----------------+---------------------+--------------------+----------------------+---+---------+-----+
|         MEM_NUM|            ADDRESS1|ADDRESS2|CALL_OWNER|        CALL_TYPE|CASE_MANAGEMENT|CASE_MGR_FIRST_NAME|CASE_MGR_LAST_NAME|            CITY|CONTRACT_CODE|COVERAGE_CODE|DATE_OF_BIRTH|DELIVERY_MODEL_ID|EMPLOYER_GRP_CODE|ENC

In [9]:
sql = '''
select 
    SUBSTRING(a.INTERACTION_DATE, 1, 10) AS yyyy_mm_dd,
        SUBSTRING(a.INTERACTION_DATE, 1, 4) AS yyyy,
        SUBSTRING(a.INTERACTION_DATE, 1, 7) AS yyyy_mm,
        SUBSTRING(a.INTERACTION_DATE, 6, 5) AS mm_dd,
        count(distinct MEM_NUM) as mem_cnt
        
from BCL_DATA a
--where SUBSTRING(a.INTERACTION_DATE, 1, 4)  = '2020'
group by
    SUBSTRING(a.INTERACTION_DATE, 1, 10),
        SUBSTRING(a.INTERACTION_DATE, 1, 4),
        SUBSTRING(a.INTERACTION_DATE, 1, 7),
        SUBSTRING(a.INTERACTION_DATE, 6, 5)

'''
df = spark.sql(sql).toPandas()
len(df),df.head()

(6271,    yyyy_mm_dd  yyyy  yyyy_mm  mm_dd  mem_cnt
 0  2002-11-16  2002  2002-11  11-16       78
 1  2009-10-19  2009  2009-10  10-19       74
 2  2017-07-08  2017  2017-07  07-08       31
 3  2005-06-01  2005  2005-06  06-01      100
 4  2009-04-14  2009  2009-04  04-14       22)

In [14]:
print(len(df[df['yyyy'] == '2017']), sum(df[df['yyyy'] == '2017']['mem_cnt']))
print(len(df[df['yyyy'] == '2018']), sum(df[df['yyyy'] == '2018']['mem_cnt']))
print(len(df[df['yyyy'] == '2019']), sum(df[df['yyyy'] == '2019']['mem_cnt']))
df[df['yyyy'] == '2019'].head()

365 14987
365 14492
150 6435


yyyy_mm_dd  yyyy  yyyy_mm  mm_dd  mem_cnt
9    2019-03-29  2019  2019-03  03-29       53
28   2019-03-17  2019  2019-03  03-17       69
46   2019-01-27  2019  2019-01  01-27       59
97   2019-04-19  2019  2019-04  04-19       30
108  2019-01-28  2019  2019-01  01-28       51